In [1]:
import pennylane as qml
from pennylane import numpy as np

# Parameters
t = 3                      # number of counting qubits
alpha = 5 / 8              # phase = 0.101 (binary)
wires = list(range(t + 1)) # counting qubits + target

dev = qml.device("default.qubit", wires=t+1, shots=1)

def controlled_U(power):
    """Returns a controlled phase gate: e^{2pi i * alpha * 2^power}"""
    angle = 2 * np.pi * alpha * (2 ** power)
    return qml.ctrl(qml.PhaseShift, control=wires[power])(angle, wires=t)

@qml.qnode(dev)
def phase_estimation_circuit():
    # Step 1: Initialize |0...0>|u> where |u> is an eigenstate
    # We'll use |1> as eigenstate of PhaseShift
    qml.PauliX(wires[t])

    # Step 2: Apply Hadamards to counting qubits
    for i in range(t):
        qml.Hadamard(wires=i)

    # Step 3: Apply controlled-U^{2^j}
    for i in range(t):
        angle = 2 * np.pi * alpha * (2 ** (t - 1 - i))
        qml.ctrl(qml.PhaseShift, control=wires[i])(angle, wires=t)

    # Step 4: Apply inverse QFT
    qml.adjoint(qml.templates.QFT)(wires=wires[:t])

    # Step 5: Measure
    return qml.sample(wires=range(t))

# Run the circuit
samples = [phase_estimation_circuit() for _ in range(10)]
for s in samples:
    print("Measured bits:", s, "=> Decimal:", int("".join(str(b) for b in s), 2))


Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
Measured bits: [1 0 1] => Decimal: 5
